In [2]:
import tensorrt
import tensorflow as tf

2023-06-17 12:47:18.491329: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Tensors and operations

In [3]:
t = tf.constant([[1.,2.,3.],[4.,5.,6.]])
t

2023-06-17 12:47:20.099217: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-17 12:47:20.265856: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-17 12:47:20.265928: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-17 12:47:20.273456: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-06-17 12:47:20.273524: I tensorflow/compile

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [4]:
display(t.shape)
display(t.dtype)

TensorShape([2, 3])

tf.float32

In [5]:
display(t[:, 1:])
display(t[..., 1, tf.newaxis])

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

In [6]:
t + 10

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [7]:
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [8]:
t @ tf.transpose(t)

2023-06-17 12:47:23.845138: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

In [9]:
# Note the shape is empty
t2 = tf.constant(42)
display(t2)
display(t2.shape)

<tf.Tensor: shape=(), dtype=int32, numpy=42>

TensorShape([])

# Tensors and Numpy

In [10]:
import numpy as np

In [11]:
a = np.array([2., 4., 5.])
print('Note: dtype=float64 is the default numpy but its unnecessary for most ANN! set dtype=tf.float32 manually!')
display(tf.constant(a, dtype=tf.float32))
display(t.numpy())
display(tf.square(a))
np.square(t)

Note: dtype=float64 is the default numpy but its unnecessary for most ANN! set dtype=tf.float32 manually!


<tf.Tensor: shape=(3,), dtype=float32, numpy=array([2., 4., 5.], dtype=float32)>

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

## Type conversions
Tensor flow does not convert types automatically! It always raises an error

In [27]:
f32 = tf.constant(2.)
print(f32.dtype)
i32 = tf.constant(40)
print(i32.dtype)
f64 = tf.constant(2., dtype=tf.float64)
print(f64.dtype)
f32 + f32

<dtype: 'float32'>
<dtype: 'int32'>
<dtype: 'float64'>


<tf.Tensor: shape=(), dtype=float32, numpy=4.0>

In [25]:
f32 + i32

InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a int32 tensor [Op:AddV2]

In [26]:
f32 + f64

InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a float tensor but is a double tensor [Op:AddV2]

In [28]:
tf.cast(f32, dtype=tf.float64) + f64

<tf.Tensor: shape=(), dtype=float64, numpy=4.0>

## Variables
The type and shape of a variable are fixed after initialization, but its value can change.
Changing variables can be done by calling assign, assign_add, or assign_sub

In [39]:
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [40]:
v1 = tf.Variable(v)
v2 = tf.Variable(v)
v3 = tf.Variable(v)
v4 = tf.Variable(v)

In [44]:
v1.assign(2 * v)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [46]:
v2[0, 1].assign(42)
v2

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[ 1., 42.,  3.],
       [ 4.,  5.,  6.]], dtype=float32)>

In [48]:
v3[:, 2].assign([0, 1])
v3

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 0.],
       [4., 5., 1.]], dtype=float32)>

In [49]:
v4.scatter_nd_update(
    indices=[[0, 0], [1, 2]], updates=[100, 200]
)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,   2.,   3.],
       [  4.,   5., 200.]], dtype=float32)>

## Other data structures

- tf.SparseTensor (class)
- tf.TensorArray (class)
- tf.RaggedTensor (class)
- tf.string (data type)
- tf.strings (module with operations)
- tf.sets (module with operations) - sets don't have a special type, they are represented as tensors
- tf.queue (module with queue data types)

## Playground

In [53]:
c = tf.constant([[1,2], [3,4]])
c[1]

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([3, 4], dtype=int32)>

In [ ]:
tf.queue.FIFOQueue